In [1]:
##import packages here
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer

# We'll use this later
from numpy import meshgrid

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## This sets the plot style
## to have a grid on a white background
sns.set_style("whitegrid")

In [2]:
sp500_df = pd.read_csv('sp500.csv', names = ['Date', 'Open', 'High', 'Low', 'Close'], header=0 )
sp500_df = sp500_df.dropna()
sp500_df['Date'] = pd.to_datetime(sp500_df['Date'])
sp500_df = sp500_df.set_index('Date')
sp500_df = sp500_df.drop(columns=['Open', 'High', 'Low'])
sp500_df.head()

,Close
Date,
2021-05-14,4173.85
2021-05-13,4112.50
2021-05-12,4063.04
2021-05-11,4152.10
2021-05-10,4188.43


In [3]:
tby_df = pd.read_csv('bond.csv', names = ['Date', 'Value'], header =0)
tby_df = tby_df.dropna()
tby_df['Date'] = pd.to_datetime(tby_df['Date'])
tby_df = tby_df.set_index('Date')
tby_df =tby_df.loc[tby_df.index[::-1]]
tby_df.head()

,Value
Date,
2021-05-06,1.58
2021-05-05,1.59
2021-05-04,1.61
2021-05-03,1.63
2021-04-30,1.65


In [4]:
ffr_df = pd.read_csv('ffr.csv', names = ['Date', 'FedFunds'], header=0)
ffr_df = ffr_df.dropna()
ffr_df['Date'] = pd.to_datetime(ffr_df['Date'])
ffr_df.set_index('Date', inplace=True)
ffr_df =ffr_df.loc[ffr_df.index[::-1]]
ffr_df.head()

,FedFunds
Date,
2021-04-01,0.07
2021-03-01,0.07
2021-02-01,0.08
2021-01-01,0.09
2020-12-01,0.09


In [5]:
fta_df = pd.read_csv('fedtotalassets.csv', names = ['Date', 'FedTotalAssets'], header=0)
fta_df = fta_df.dropna()
fta_df['Date'] = pd.to_datetime(fta_df['Date'])
fta_df.set_index('Date', inplace=True)
fta_df =fta_df.loc[fta_df.index[::-1]]
fta_df.head()

,FedTotalAssets
Date,
2021-05-12,7830663.0
2021-05-05,7810486.0
2021-04-28,7780962.0
2021-04-21,7820948.0
2021-04-14,7793104.0


In [6]:
eps_df =pd.read_csv('sp500eps.csv', names =['Date', 'Value'], header =0)
eps_df = eps_df.dropna()
eps_df['Date'] = pd.to_datetime(eps_df['Date'])
eps_df.set_index('Date', inplace=True)
eps_df.head()

,Value
Date,
2020-12-31,95.72
2020-11-30,97.20
2020-10-31,98.53
2020-09-30,99.95
2020-08-31,100.44


In [7]:
div_df = pd.read_csv('sp500div.csv', names=['Date', 'Value'], header=0)
div_df = div_df.dropna()
div_df['Date'] = pd.to_datetime(div_df['Date']) 
div_df.set_index('Date', inplace=True)
div_df.head()

,Value
Date,
2021-05-31,1.37
2021-04-30,1.41
2021-04-01,1.45
2021-03-31,1.47
2021-03-01,1.49


In [8]:
# vix_df = pd.read_csv('VIX.csv')
# vix_df = vix_df.rename(columns={'Trade Date': 'Date'})
# vix_df['Date'] = pd.to_datetime(vix_df['Date'])
# vix_df = vix_df.set_index('Date')
# vix_df

In [8]:
# drop rows of df2 whose date are not in df1
# def drop_date(df1, df2):
#     for i in df2.index:
#         if i not in df1.index:
#             df2 = df2.drop(i)

In [80]:
def last_day_of_month_next_month(d):
    
    return d.replace(day=28)+datetime.timedelta(days=4)

last_day_of_month_next_month(datetime.datetime(2002,12,18))


datetime.datetime(2003, 1, 1, 0, 0)

In [88]:
datetime.datetime(2002,2,1).replace(month=3)

datetime.datetime(2002, 3, 1, 0, 0)

In [93]:
import datetime as datetime
def last_day_of_month(d):
    next_month = d.replace(day=28)+datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def first_day_of_next_month(d):
    dt = last_day_of_month(d)
    return dt + datetime.timedelta(days=1)

def end_date_calc(start_date, str1):
    # starts the first day of month till the frist day of the next month
    if (str1=='ffr'):
        end_date = first_day_of_next_month(start_date.date())
    # updates weekly
    if (str1 == 'fta'):
        end_date = start_date + pd.Timedelta(days=7)
    # starts at the end of month till the end of the next month
    if (str1 == 'eps'):
        end_date = last_day_of_month(start_date.date()+datetime.timedelta(days=1))
    # if next index ends with 01, then starts at 01 till the end of month
    if (str1 == 'div'):
        end_date = last_day_of_month(start_date.date()+ datetime.timedelta(days=1))
    return end_date    

def convert_to_daily(df, df2, str1, str2 ):
    for i in df2.index: 
        start_date = i 
        
        if (start_date + pd.Timedelta(days=1) in  df2.index):
            continue
        
        end_date = end_date_calc(start_date, str1 )
        if end_date <= df.index[-1]:
            continue
        delta = datetime.timedelta(days=1)      
        while start_date < end_date:
            
            if start_date in df.index:
                df.loc[start_date, str1] = df2[str2].loc[i]  
        
            start_date += delta 

In [96]:
convert_to_daily(df, div_df2, 'div', 'Value' )
df['div'].tail(50)

Date
2003-03-03    1.93
2003-02-28    1.93
2003-02-27    1.80
2003-02-26    1.80
2003-02-25    1.80
2003-02-24    1.80
2003-02-21    1.80
2003-02-20    1.80
2003-02-19    1.80
2003-02-18    1.80
2003-02-14    1.80
2003-02-13    1.80
2003-02-12    1.80
2003-02-11    1.80
2003-02-10    1.80
2003-02-07    1.80
2003-02-06    1.80
2003-02-05    1.80
2003-02-04    1.80
2003-02-03    1.80
2003-01-31    1.80
2003-01-30    1.79
2003-01-29    1.79
2003-01-28    1.79
2003-01-27    1.79
2003-01-24    1.79
2003-01-23    1.79
2003-01-22    1.79
2003-01-21    1.79
2003-01-17    1.79
2003-01-16    1.79
2003-01-15    1.79
2003-01-14    1.79
2003-01-13    1.79
2003-01-10    1.79
2003-01-09    1.79
2003-01-08    1.79
2003-01-07    1.79
2003-01-06    1.79
2003-01-03    1.79
2003-01-02    1.79
2002-12-31    1.79
2002-12-30     NaN
2002-12-27     NaN
2002-12-26     NaN
2002-12-24     NaN
2002-12-23     NaN
2002-12-20     NaN
2002-12-19     NaN
2002-12-18     NaN
Name: div, dtype: float64

In [97]:
from dateutil.relativedelta import relativedelta
sp500_df_min = sp500_df.index[np.argmin(sp500_df.index)]
sp500_df_max = sp500_df.index[np.argmax(sp500_df.index)]

tby_df_min = tby_df.index[np.argmin(tby_df.index)]
tby_df_max = tby_df.index[np.argmax(tby_df.index)]

ffr_df_min = ffr_df.index[np.argmin(ffr_df.index)]
ffr_df_max = ffr_df.index[np.argmax(ffr_df.index)] + relativedelta(months=1)

fta_df_min = fta_df.index[np.argmin(fta_df.index)]
fta_df_max = fta_df.index[np.argmax(fta_df.index)] + pd.Timedelta(weeks=1)

eps_df_min = eps_df.index[np.argmin(eps_df.index)] 
eps_df_max = eps_df.index[np.argmax(eps_df.index)] + relativedelta(months=1)

div_df_min = div_df.index[np.argmin(div_df.index)]
div_df_max = div_df.index[np.argmax(div_df.index)] + relativedelta(months=1)

In [98]:
min_date = max(sp500_df_min, tby_df_min, ffr_df_min,fta_df_min ,eps_df_min , div_df_min)
max_date = min(sp500_df_max,tby_df_max,ffr_df_max,fta_df_max, eps_df_max,div_df_max )
print(min_date, max_date)

2002-12-18 00:00:00 2021-01-31 00:00:00


In [99]:
def date_range(df):
    return [x for x in df.index if x <= max_date and x >= min_date]


df = sp500_df.loc[date_range(sp500_df), ].copy()
df['tby'] = tby_df['Value'] 
df['ffr'] = ffr_df['FedFunds']
df['fta'] = fta_df['FedTotalAssets']
df['eps'] = eps_df['Value']
df['div'] = div_df['Value']

In [100]:
ffr_df2 = ffr_df.loc[date_range(ffr_df), ].copy()
fta_df2 = fta_df.loc[date_range(fta_df), ].copy()
eps_df2 = eps_df.loc[date_range(eps_df), ].copy()
div_df2 = div_df.loc[date_range(div_df), ].copy()

In [104]:
convert_to_daily(df, ffr_df2, 'ffr', 'FedFunds' )
convert_to_daily(df, fta_df2, 'fta', 'FedTotalAssets' )
convert_to_daily(df, eps_df2, 'eps', 'Value' )
convert_to_daily(df, div_df2, 'div', 'Value' )
df =df.loc[df.index[::-1]]
df = df.dropna()
df

,Close,tby,ffr,fta,eps,div
Date,,,,,,
2003-01-02,909.03,4.07,1.24,732202.0,40.40,1.79
2003-01-03,908.59,4.05,1.24,732202.0,40.40,1.79
2003-01-06,929.01,4.09,1.24,732202.0,40.40,1.79
2003-01-07,922.93,4.04,1.24,732202.0,40.40,1.79
2003-01-08,909.93,4.00,1.24,724902.0,40.40,1.79
...,...,...,...,...,...,...
2021-01-25,3855.36,1.05,0.09,7414942.0,95.72,1.58
2021-01-26,3849.62,1.05,0.09,7414942.0,95.72,1.58
2021-01-27,3750.77,1.04,0.09,7404926.0,95.72,1.58


In [105]:
df.to_csv(r'/Users/yuqingdai/Documents/GitHub/Stock-Erdos/scratch work/Yuqing-Data-Merge/df4.csv')

In [15]:
import datetime as datetime
df = sp500_df.copy()

drop_date(df,tby_df)
drop_date(df,ffr_df)
drop_date(df,fta_df)
drop_date(df,eps_df)
drop_date(df,div_df)

print(df.index)

df['tby'] = tby_df['Value'] 
df['ffr'] = ffr_df['FedFunds']
df['fta'] = fta_df['FedTotalAssets']
df['eps'] = eps_df['Value']
df['div'] = div_df['Value']

convert_to_daily(df, ffr_df, 'ffr', 'FedFunds' )
convert_to_daily(df, fta_df, 'fta', 'FedTotalAssets' )
convert_to_daily(df, eps_df, 'eps', 'Value' )
convert_to_daily(df, div_df, 'div', 'Value' )

# df = df.dropna()
# df =df.loc[df.index[::-1]]
df = df.dropna()
df

DatetimeIndex(['2021-05-14', '2021-05-13', '2021-05-12', '2021-05-11',
               '2021-05-10', '2021-05-07', '2021-05-06', '2021-05-05',
               '2021-05-04', '2021-05-03',
               ...
               '1978-01-16', '1978-01-13', '1978-01-12', '1978-01-11',
               '1978-01-10', '1978-01-09', '1978-01-06', '1978-01-05',
               '1978-01-04', '1978-01-03'],
              dtype='datetime64[ns]', name='Date', length=10936, freq=None)
2021-04-01 00:00:00
2021-03-01 00:00:00
2021-02-01 00:00:00
2021-01-01 00:00:00
2020-12-01 00:00:00
2020-11-01 00:00:00
2020-10-01 00:00:00
2020-09-01 00:00:00
2020-08-01 00:00:00
2020-07-01 00:00:00
2020-06-01 00:00:00
2020-05-01 00:00:00
2020-04-01 00:00:00
2020-03-01 00:00:00
2020-02-01 00:00:00
2020-01-01 00:00:00
2019-12-01 00:00:00
2019-11-01 00:00:00
2019-10-01 00:00:00
2019-09-01 00:00:00
2019-08-01 00:00:00
2019-07-01 00:00:00
2019-06-01 00:00:00
2019-05-01 00:00:00
2019-04-01 00:00:00
2019-03-01 00:00:00
2019-02-01 00:

1986-01-01 00:00:00
1985-12-01 00:00:00
1985-11-01 00:00:00
1985-10-01 00:00:00
1985-09-01 00:00:00
1985-08-01 00:00:00
1985-07-01 00:00:00
1985-06-01 00:00:00
1985-05-01 00:00:00
1985-04-01 00:00:00
1985-03-01 00:00:00
1985-02-01 00:00:00
1985-01-01 00:00:00
1984-12-01 00:00:00
1984-11-01 00:00:00
1984-10-01 00:00:00
1984-09-01 00:00:00
1984-08-01 00:00:00
1984-07-01 00:00:00
1984-06-01 00:00:00
1984-05-01 00:00:00
1984-04-01 00:00:00
1984-03-01 00:00:00
1984-02-01 00:00:00
1984-01-01 00:00:00
1983-12-01 00:00:00
1983-11-01 00:00:00
1983-10-01 00:00:00
1983-09-01 00:00:00
1983-08-01 00:00:00
1983-07-01 00:00:00
1983-06-01 00:00:00
1983-05-01 00:00:00
1983-04-01 00:00:00
1983-03-01 00:00:00
1983-02-01 00:00:00
1983-01-01 00:00:00
1982-12-01 00:00:00
1982-11-01 00:00:00
1982-10-01 00:00:00
1982-09-01 00:00:00
1982-08-01 00:00:00
1982-07-01 00:00:00
1982-06-01 00:00:00
1982-05-01 00:00:00
1982-04-01 00:00:00
1982-03-01 00:00:00
1982-02-01 00:00:00
1982-01-01 00:00:00
1981-12-01 00:00:00


2019-10-09 00:00:00
2019-10-02 00:00:00
2019-09-25 00:00:00
2019-09-18 00:00:00
2019-09-11 00:00:00
2019-09-04 00:00:00
2019-08-28 00:00:00
2019-08-21 00:00:00
2019-08-14 00:00:00
2019-08-07 00:00:00
2019-07-31 00:00:00
2019-07-24 00:00:00
2019-07-17 00:00:00
2019-07-10 00:00:00
2019-07-03 00:00:00
2019-06-26 00:00:00
2019-06-19 00:00:00
2019-06-12 00:00:00
2019-06-05 00:00:00
2019-05-29 00:00:00
2019-05-22 00:00:00
2019-05-15 00:00:00
2019-05-08 00:00:00
2019-05-01 00:00:00
2019-04-24 00:00:00
2019-04-17 00:00:00
2019-04-10 00:00:00
2019-04-03 00:00:00
2019-03-27 00:00:00
2019-03-20 00:00:00
2019-03-13 00:00:00
2019-03-06 00:00:00
2019-02-27 00:00:00
2019-02-20 00:00:00
2019-02-13 00:00:00
2019-02-06 00:00:00
2019-01-30 00:00:00
2019-01-23 00:00:00
2019-01-16 00:00:00
2019-01-09 00:00:00
2019-01-02 00:00:00
2018-12-26 00:00:00
2018-12-19 00:00:00
2018-12-12 00:00:00
2018-12-05 00:00:00
2018-11-28 00:00:00
2018-11-21 00:00:00
2018-11-14 00:00:00
2018-11-07 00:00:00
2018-10-31 00:00:00


2007-11-28 00:00:00
2007-11-21 00:00:00
2007-11-14 00:00:00
2007-11-07 00:00:00
2007-10-31 00:00:00
2007-10-24 00:00:00
2007-10-17 00:00:00
2007-10-10 00:00:00
2007-10-03 00:00:00
2007-09-26 00:00:00
2007-09-19 00:00:00
2007-09-12 00:00:00
2007-09-05 00:00:00
2007-08-29 00:00:00
2007-08-22 00:00:00
2007-08-15 00:00:00
2007-08-08 00:00:00
2007-08-01 00:00:00
2007-07-25 00:00:00
2007-07-18 00:00:00
2007-07-11 00:00:00
2007-07-04 00:00:00
2007-06-27 00:00:00
2007-06-20 00:00:00
2007-06-13 00:00:00
2007-06-06 00:00:00
2007-05-30 00:00:00
2007-05-23 00:00:00
2007-05-16 00:00:00
2007-05-09 00:00:00
2007-05-02 00:00:00
2007-04-25 00:00:00
2007-04-18 00:00:00
2007-04-11 00:00:00
2007-04-04 00:00:00
2007-03-28 00:00:00
2007-03-21 00:00:00
2007-03-14 00:00:00
2007-03-07 00:00:00
2007-02-28 00:00:00
2007-02-21 00:00:00
2007-02-14 00:00:00
2007-02-07 00:00:00
2007-01-31 00:00:00
2007-01-24 00:00:00
2007-01-17 00:00:00
2007-01-10 00:00:00
2007-01-03 00:00:00
2006-12-27 00:00:00
2006-12-20 00:00:00


2005-06-30 00:00:00
2005-05-31 00:00:00
2005-04-30 00:00:00
2005-03-31 00:00:00
2005-02-28 00:00:00
2005-01-31 00:00:00
2004-12-31 00:00:00
2004-11-30 00:00:00
2004-10-31 00:00:00
2004-09-30 00:00:00
2004-08-31 00:00:00
2004-07-31 00:00:00
2004-06-30 00:00:00
2004-05-31 00:00:00
2004-04-30 00:00:00
2004-03-31 00:00:00
2004-02-29 00:00:00
2004-01-31 00:00:00
2003-12-31 00:00:00
2003-11-30 00:00:00
2003-10-31 00:00:00
2003-09-30 00:00:00
2003-08-31 00:00:00
2003-07-31 00:00:00
2003-06-30 00:00:00
2003-05-31 00:00:00
2003-04-30 00:00:00
2003-03-31 00:00:00
2003-02-28 00:00:00
2003-01-31 00:00:00
2002-12-31 00:00:00
2002-11-30 00:00:00
2002-10-31 00:00:00
2002-09-30 00:00:00
2002-08-31 00:00:00
2002-07-31 00:00:00
2002-06-30 00:00:00
2002-05-31 00:00:00
2002-04-30 00:00:00
2002-03-31 00:00:00
2002-02-28 00:00:00
2002-01-31 00:00:00
2001-12-31 00:00:00
2001-11-30 00:00:00
2001-10-31 00:00:00
2001-09-30 00:00:00
2001-08-31 00:00:00
2001-07-31 00:00:00
2001-06-30 00:00:00
2001-05-31 00:00:00


1904-05-31 00:00:00
1904-04-30 00:00:00
1904-03-31 00:00:00
1904-02-29 00:00:00
1904-01-31 00:00:00
1903-12-31 00:00:00
1903-11-30 00:00:00
1903-10-31 00:00:00
1903-09-30 00:00:00
1903-08-31 00:00:00
1903-07-31 00:00:00
1903-06-30 00:00:00
1903-05-31 00:00:00
1903-04-30 00:00:00
1903-03-31 00:00:00
1903-02-28 00:00:00
1903-01-31 00:00:00
1902-12-31 00:00:00
1902-11-30 00:00:00
1902-10-31 00:00:00
1902-09-30 00:00:00
1902-08-31 00:00:00
1902-07-31 00:00:00
1902-06-30 00:00:00
1902-05-31 00:00:00
1902-04-30 00:00:00
1902-03-31 00:00:00
1902-02-28 00:00:00
1902-01-31 00:00:00
1901-12-31 00:00:00
1901-11-30 00:00:00
1901-10-31 00:00:00
1901-09-30 00:00:00
1901-08-31 00:00:00
1901-07-31 00:00:00
1901-06-30 00:00:00
1901-05-31 00:00:00
1901-04-30 00:00:00
1901-03-31 00:00:00
1901-02-28 00:00:00
1901-01-31 00:00:00
1900-12-31 00:00:00
1900-11-30 00:00:00
1900-10-31 00:00:00
1900-09-30 00:00:00
1900-08-31 00:00:00
1900-07-31 00:00:00
1900-06-30 00:00:00
1900-05-31 00:00:00
1900-04-30 00:00:00


1972-07-31 00:00:00
1972-06-30 00:00:00
1972-05-31 00:00:00
1972-04-30 00:00:00
1972-03-31 00:00:00
1972-02-29 00:00:00
1972-01-31 00:00:00
1971-12-31 00:00:00
1971-11-30 00:00:00
1971-10-31 00:00:00
1971-09-30 00:00:00
1971-08-31 00:00:00
1971-07-31 00:00:00
1971-06-30 00:00:00
1971-05-31 00:00:00
1971-04-30 00:00:00
1971-03-31 00:00:00
1971-02-28 00:00:00
1971-01-31 00:00:00
1970-12-31 00:00:00
1970-11-30 00:00:00
1970-10-31 00:00:00
1970-09-30 00:00:00
1970-08-31 00:00:00
1970-07-31 00:00:00
1970-06-30 00:00:00
1970-05-31 00:00:00
1970-04-30 00:00:00
1970-03-31 00:00:00
1970-02-28 00:00:00
1970-01-31 00:00:00
1969-12-31 00:00:00
1969-11-30 00:00:00
1969-10-31 00:00:00
1969-09-30 00:00:00
1969-08-31 00:00:00
1969-07-31 00:00:00
1969-06-30 00:00:00
1969-05-31 00:00:00
1969-04-30 00:00:00
1969-03-31 00:00:00
1969-02-28 00:00:00
1969-01-31 00:00:00
1968-12-31 00:00:00
1968-11-30 00:00:00
1968-10-31 00:00:00
1968-09-30 00:00:00
1968-08-31 00:00:00
1968-07-31 00:00:00
1968-06-30 00:00:00


1889-03-31 00:00:00
1889-02-28 00:00:00
1889-01-31 00:00:00
1888-12-31 00:00:00
1888-11-30 00:00:00
1888-10-31 00:00:00
1888-09-30 00:00:00
1888-08-31 00:00:00
1888-07-31 00:00:00
1888-06-30 00:00:00
1888-05-31 00:00:00
1888-04-30 00:00:00
1888-03-31 00:00:00
1888-02-29 00:00:00
1888-01-31 00:00:00
1887-12-31 00:00:00
1887-11-30 00:00:00
1887-10-31 00:00:00
1887-09-30 00:00:00
1887-08-31 00:00:00
1887-07-31 00:00:00
1887-06-30 00:00:00
1887-05-31 00:00:00
1887-04-30 00:00:00
1887-03-31 00:00:00
1887-02-28 00:00:00
1887-01-31 00:00:00
1886-12-31 00:00:00
1886-11-30 00:00:00
1886-10-31 00:00:00
1886-09-30 00:00:00
1886-08-31 00:00:00
1886-07-31 00:00:00
1886-06-30 00:00:00
1886-05-31 00:00:00
1886-04-30 00:00:00
1886-03-31 00:00:00
1886-02-28 00:00:00
1886-01-31 00:00:00
1885-12-31 00:00:00
1885-11-30 00:00:00
1885-10-31 00:00:00
1885-09-30 00:00:00
1885-08-31 00:00:00
1885-07-31 00:00:00
1885-06-30 00:00:00
1885-05-31 00:00:00
1885-04-30 00:00:00
1885-03-31 00:00:00
1885-02-28 00:00:00


,Close,tby,ffr,fta,eps,div
Date,,,,,,
2020-12-31,3756.07,0.93,0.09,7363351.0,95.72,1.58
2020-12-30,3732.04,0.93,0.09,7363351.0,97.20,1.61
2020-12-29,3727.04,0.94,0.09,7404039.0,97.20,1.61
2020-12-28,3735.36,0.94,0.09,7404039.0,97.20,1.61
2020-12-24,3703.06,0.94,0.09,7404039.0,97.20,1.61
...,...,...,...,...,...,...
2003-04-30,916.92,3.89,1.26,747460.0,45.73,1.82
2003-03-31,848.18,3.83,1.25,726250.0,43.60,1.92
2003-02-28,841.15,3.71,1.26,723222.0,42.55,1.93


In [13]:
# non-periodic
# def convert_to_daily(df, df2, str1, str2 ):
#     i = len(df2.index)-1
#     while i >0:
#         start_date = df2.index[i]

#         if (start_date + pd.Timedelta(days=1) in  df2.index):
#             continue
        
#         end_date = df2.index[i-1]
#         if end_date <= df.index[-1]:
#             continue
            
            
#         delta = datetime.timedelta(days=1)      
#         while start_date < end_date:
#             if start_date in df.index:
#                 df.loc[start_date, str1] = df2[str2].loc[df2.index[i]]  
        
#             start_date += delta 
        
#         i -= 1

# import datetime as datetime
# df = sp500_df.copy()
# drop_date(df,div_df)
# df['div'] = div_df['Value']

# convert_to_daily(df, div_df, 'div', 'Value' )
# df.dropna()
# df